<a href="https://colab.research.google.com/github/rajat98v/Sign-Language-Recognition/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To download dataset from Kaggle
## Get API Token
1. Make Kaggle Account at https://kaggle.com
2. Go to Profile -> Accounts -> API -> Click Create New API Token
3. Download the kaggle.json file 


Install Kaggle API

In [ ]:
 ! pip install -q kaggle

Upload the kaggle.json file to colab

In [ ]:
 from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rajat98v","key":"3bd74e6f20262cf73f5ed4f5c6ba5bf5"}'}

Create Config file directory for kaggle & upload kaggle.json (TOKEN).

In [ ]:
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list  # List all the datasets available

#### Download Asl-alphabet database
https://www.kaggle.com/grassknoted/asl-alphabet

In [ ]:
! kaggle datasets download -d grassknoted/asl-alphabet

 99% 1.01G/1.03G [00:11<00:00, 138MB/s]
100% 1.03G/1.03G [00:11<00:00, 94.7MB/s]


In [ ]:
!pwd  # all the shell command is executing under this directory by default

/content


In [ ]:
!unzip asl-alphabet.zip # unzip the database

# Resourse
## Code
- https://www.kaggle.com/serkanpeldek/cnn-practices-on-sign-language-digits (64X64)

## Dataset
- https://www.kaggle.com/grassknoted/asl-alphabet (200X200)

The training data set contains 87,000 images which are 200x200 pixels. There are 29 classes, of which 26 are for the letters A-Z and 3 classes for SPACE, DELETE and NOTHING.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(215, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(29, activation='softmax')  
])


In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


TRAINING_DIR = "/content/asl_alphabet_train/asl_alphabet_train"
image_generator = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

train_generator = image_generator.flow_from_directory(
	TRAINING_DIR,
	target_size=(200,200),
	class_mode='categorical',
  batch_size=126
)

validation_generator = image_generator.flow_from_directory(
	TRAINING_DIR,
	target_size=(200,200),
	class_mode='categorical',
  batch_size=126
)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=5,
                              validation_steps=50,
                              verbose=1)

In [ ]:
model.save("rps.h5")

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()


In [ ]:


import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(200, 200,3))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

